## NMF(Non-negative matrix factorization)

In [4]:
# !pip install pymysql
# !pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 KB 3.1 MB/s eta 0:00:00


In [24]:
import os
import pymysql
import time
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
import json

In [25]:
# 파일 경로 설정
file_path = '../train_data/Data5_train_final.csv'  # 파일 경로를 적절히 변경하세요

# pd.read_csv와 반복문을 사용하여 파일을 나누어 읽기
chunks = pd.read_csv(file_path, encoding='utf-8', chunksize=100)

# past_data라는 빈 데이터프레임 생성
past_data = pd.DataFrame()

for chunk in chunks:
    # 각 청크에 대한 처리를 여기에 추가
    # 예를 들어, 원하는 계산 수행 또는 데이터 전처리 등을 수행

    # past_data에 현재 청크를 추가
    past_data = pd.concat([past_data, chunk])

# past_data를 사용하여 나머지 작업 수행
print(past_data.head())  # past_data의 처음 몇 행을 출력하는 예시


  musical_id   title                                         poster_url genre  \
0       3885    실연남녀  http://ticketimage.interpark.com/PlayDictionar...    창작   
1       3635  김종욱 찾기  http://ticketimage.interpark.com/PlayDictionar...    창작   
2       4829     첫사랑  http://ticketimage.interpark.com/PlayDictionar...    창작   
3       3322     미라클  http://ticketimage.interpark.com/PlayDictionar...    창작   
4       4424     대장금  http://ticketimage.interpark.com/PlayDictionar...    창작   

                      date  start_date    end_date     location  \
0  2007/10/13 ~ 2007/12/09  2007-10-13  2007-12-09   대학로 티오엠 1관   
1  2006/12/12 ~ 2007/04/08  2006-12-12  2007-04-08   JTN 아트홀 1관   
2  2007/03/27 ~ 2007/06/17  2007-03-27  2007-06-17   대학로 티오엠 1관   
3  2007/06/01 ~ 2013/02/17  2007-06-01  2013-02-17   미라클 씨어터 1관   
4  2007/05/26 ~ 2007/06/17  2007-05-26  2007-06-17  예술의전당 오페라극장   

                                              actors age_rating  ...  \
0                      엄기준,신성록,강지우,양소민

In [26]:
past_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,...,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector,synopsis_numpy,synopsis_numpy_scale,tag1,tag2,tag3
0,3885,실연남녀,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/10/13 ~ 2007/12/09,2007-10-13,2007-12-09,대학로 티오엠 1관,"엄기준,신성록,강지우,양소민,한애리,변우민,성민...",7세이상,...,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"['조폭', '형제', '경영', '아트', '퓨쳐', '산장', '이', '개월'...",b'\x0e\x11u<\xdeM1=\xa1kX=\xd11\x06\xbe?\x03\x...,[ 0.01495768 0.04328715 0.05283702 -0.131049...,"[-0.6805413961410522, 0.3137694299221039, -0.2...",사람,사랑,슬픔
1,3635,김종욱 찾기,http://ticketimage.interpark.com/PlayDictionar...,창작,2006/12/12 ~ 2007/04/08,2006-12-12,2007-04-08,JTN 아트홀 1관,"신성록,전병욱,오나라,안유진,원종환,김재범,김지현...",만 13세이상,...,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"['첫사랑', '나선', '여자', '첫사랑', '남자', '티격태격', '로맨틱'...",b'\xddC\xf5=I@Z=%\xfd\x8b\xbc\xd8\x03==\xd6q\x...,[ 0.11975835 0.05328396 -0.01708848 0.046146...,"[0.03893159702420235, 0.3932909667491913, -0.7...",사랑,오페라,NaN
2,4829,첫사랑,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/03/27 ~ 2007/06/17,2007-03-27,2007-06-17,대학로 티오엠 1관,"조정석,홍광호,해이,전경수,이정섭,김성기,김인수...",만 6세이상,...,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"['시간', '조금', '수', '있다', '적', '동네', '해수', '오누이'...","b""S\xa2\xa0<\xeb\xad\xd5<\xd6\x93\xbe\xbd\x01\...",[ 0.01960865 0.02608391 -0.09305541 -0.089630...,"[-0.6486117243766785, 0.176923006772995, -1.25...",오페라,NaN,NaN
3,3322,미라클,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/06/01 ~ 2013/02/17,2007-06-01,2013-02-17,미라클 씨어터 1관,"신현묵,조호균,김고운,원혜지,남궁인,윤도훈,김학규...",만 12세이상,...,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"['인기', '그룹', '핫바', '멤버', '가수', '희다', '은', '날',...",b'^\xb5D>\xb63\xad=P\x8e >.\x97\x02\xbe\xee+\x...,[ 1.92098111e-01 8.45712870e-02 1.56792879e-...,"[0.5355554223060608, 0.6421719193458557, 0.540...",오페라,NaN,NaN
4,4424,대장금,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/05/26 ~ 2007/06/17,2007-05-26,2007-06-17,예술의전당 오페라극장,"최보영,안유진,김우형,김소현,이태원,류수화,손광업,한애리,한성식,김나윤,김일우,최윤...",만 7세이상,...,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"['어머니', '유언', '위해', '궁', '생각', '시가', '장금', '것'...",b'@\x8a\xa8=\x8e\xd7\xe0=\x1e\x10\x9b=N%\xf6\x...,[ 0.08229494 0.10978614 0.07571433 -0.120188...,"[-0.21826054155826569, 0.84274822473526, -0.04...",사랑,오페라,아이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11982,150211,브레멘 음악대 부산,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2019/11/16 ~ 2019/11/17,2019-11-16,2019-11-17,신세계센텀시티 문화홀9층,NaN,24개월이상,...,NaN,집에서 키워지던 동물들이 주인의 천대를 받고쫓겨나거나 스스로의 자유를 위해 집을 나...,집 동물 주인 천대 스스로 자유 위해 집 친구 멋지다 악기 음악회 만들기 즐겁다 시...,"['집', '동물', '주인', '천대', '스스로', '자유', '위해', '집'...","b'e#^\xbc\xc5o\x10\xbd\x07,\x02>\xa4\xd4\x1c>\...",[-1.35582434e-02 -3.52628417e-02 1.27121076e-...,"[-0.8763076663017273, -0.3110712766647339, 0.3...",친구,예술,NaN
11983,150242,헤이지니 럭키강이 2 청주,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2019/12/28 ~ 2019/12/29,2019-12-28,2019-12-29,청주예술의전당 대공연장,NaN,24개월이상,...,NaN,딩동 마법의 세계로 가야할 택배상자가 지니강이 집으로 배달되었다신비로운 마법의 열쇠...,딩동 마법 세계 가야 택배 상자 지니 강 집 배달 신비롭다 마법 열쇠 지니 강 비밀...,"['딩동', '마법', '세계', '가야', '택배', '상자', '지니', '강'...","b""\xe1uF=`\x1c\xb3<\x01\x92\x93>\xfc&~\xbe|\xa...",[ 0.04845226 0.02186412 0.2882233 -0.248195...,"[-0.4505958557128906, 0.14335589110851288, 1.4...",오페라,마법,NaN
11984,150349,난타 대구,http://ticketimage.interpark.com/PlayDictionar...,퍼포먼스,2019/12/20 ~ 201

In [27]:
# musical_id를 기준으로 중복된 행 찾기
duplicate_rows = past_data[past_data.duplicated(subset='musical_id', keep=False)]
print(duplicate_rows)

                                             musical_id     title  \
867                                                7962  난타  강남공연   
868                                                7962  난타  강남공연   
869                                                7962  난타  강남공연   
870                                                7962  난타  강남공연   
871                                                7962  난타  강남공연   
...                                                 ...       ...   
7295     톰이 친구들을 초대해 댄스파티를 여니 낮잠을 자던 제리에겐 여간 시끄러운 소동...       NaN   
7296     톰이 친구들을 초대해 댄스파티를 여니 낮잠을 자던 제리에겐 여간 시끄러운 소동...       NaN   
7297     톰이 친구들을 초대해 댄스파티를 여니 낮잠을 자던 제리에겐 여간 시끄러운 소동...       NaN   
7298     톰이 친구들을 초대해 댄스파티를 여니 낮잠을 자던 제리에겐 여간 시끄러운 소동...       NaN   
7299     톰이 친구들을 초대해 댄스파티를 여니 낮잠을 자던 제리에겐 여간 시끄러운 소동...       NaN   

                                             poster_url genre  \
867   http://ticketimage.interpark.com/PlayDictionar...    창작   
868   http://ticketimage.interpark.com/Pl

In [28]:
# 'musical_id' 열의 값이 숫자로만 이루어진 행만 남기기
past_data = past_data[past_data['musical_id'].apply(lambda x: str(x).isdigit())]

In [29]:
# musical_id를 기준으로 중복된 행 중 첫 번째 행만 남기고 나머지 중복 행 삭제
past_data.drop_duplicates(subset='musical_id', keep='first', inplace=True)

/tmp/ipykernel_21150/3904138153.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  past_data.drop_duplicates(subset='musical_id', keep='first', inplace=True)


In [30]:
# past_data = pd.read_csv('../train_data/Data5_train_final.csv', encoding='utf-8')# 데이터 파일 경로
past_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,...,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector,synopsis_numpy,synopsis_numpy_scale,tag1,tag2,tag3
0,3885,실연남녀,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/10/13 ~ 2007/12/09,2007-10-13,2007-12-09,대학로 티오엠 1관,"엄기준,신성록,강지우,양소민,한애리,변우민,성민...",7세이상,...,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"['조폭', '형제', '경영', '아트', '퓨쳐', '산장', '이', '개월'...",b'\x0e\x11u<\xdeM1=\xa1kX=\xd11\x06\xbe?\x03\x...,[ 0.01495768 0.04328715 0.05283702 -0.131049...,"[-0.6805413961410522, 0.3137694299221039, -0.2...",사람,사랑,슬픔
1,3635,김종욱 찾기,http://ticketimage.interpark.com/PlayDictionar...,창작,2006/12/12 ~ 2007/04/08,2006-12-12,2007-04-08,JTN 아트홀 1관,"신성록,전병욱,오나라,안유진,원종환,김재범,김지현...",만 13세이상,...,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"['첫사랑', '나선', '여자', '첫사랑', '남자', '티격태격', '로맨틱'...",b'\xddC\xf5=I@Z=%\xfd\x8b\xbc\xd8\x03==\xd6q\x...,[ 0.11975835 0.05328396 -0.01708848 0.046146...,"[0.03893159702420235, 0.3932909667491913, -0.7...",사랑,오페라,NaN
2,4829,첫사랑,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/03/27 ~ 2007/06/17,2007-03-27,2007-06-17,대학로 티오엠 1관,"조정석,홍광호,해이,전경수,이정섭,김성기,김인수...",만 6세이상,...,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"['시간', '조금', '수', '있다', '적', '동네', '해수', '오누이'...","b""S\xa2\xa0<\xeb\xad\xd5<\xd6\x93\xbe\xbd\x01\...",[ 0.01960865 0.02608391 -0.09305541 -0.089630...,"[-0.6486117243766785, 0.176923006772995, -1.25...",오페라,NaN,NaN
3,3322,미라클,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/06/01 ~ 2013/02/17,2007-06-01,2013-02-17,미라클 씨어터 1관,"신현묵,조호균,김고운,원혜지,남궁인,윤도훈,김학규...",만 12세이상,...,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"['인기', '그룹', '핫바', '멤버', '가수', '희다', '은', '날',...",b'^\xb5D>\xb63\xad=P\x8e >.\x97\x02\xbe\xee+\x...,[ 1.92098111e-01 8.45712870e-02 1.56792879e-...,"[0.5355554223060608, 0.6421719193458557, 0.540...",오페라,NaN,NaN
4,4424,대장금,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/05/26 ~ 2007/06/17,2007-05-26,2007-06-17,예술의전당 오페라극장,"최보영,안유진,김우형,김소현,이태원,류수화,손광업,한애리,한성식,김나윤,김일우,최윤...",만 7세이상,...,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"['어머니', '유언', '위해', '궁', '생각', '시가', '장금', '것'...",b'@\x8a\xa8=\x8e\xd7\xe0=\x1e\x10\x9b=N%\xf6\x...,[ 0.08229494 0.10978614 0.07571433 -0.120188...,"[-0.21826054155826569, 0.84274822473526, -0.04...",사랑,오페라,아이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11982,150211,브레멘 음악대 부산,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2019/11/16 ~ 2019/11/17,2019-11-16,2019-11-17,신세계센텀시티 문화홀9층,NaN,24개월이상,...,NaN,집에서 키워지던 동물들이 주인의 천대를 받고쫓겨나거나 스스로의 자유를 위해 집을 나...,집 동물 주인 천대 스스로 자유 위해 집 친구 멋지다 악기 음악회 만들기 즐겁다 시...,"['집', '동물', '주인', '천대', '스스로', '자유', '위해', '집'...","b'e#^\xbc\xc5o\x10\xbd\x07,\x02>\xa4\xd4\x1c>\...",[-1.35582434e-02 -3.52628417e-02 1.27121076e-...,"[-0.8763076663017273, -0.3110712766647339, 0.3...",친구,예술,NaN
11983,150242,헤이지니 럭키강이 2 청주,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2019/12/28 ~ 2019/12/29,2019-12-28,2019-12-29,청주예술의전당 대공연장,NaN,24개월이상,...,NaN,딩동 마법의 세계로 가야할 택배상자가 지니강이 집으로 배달되었다신비로운 마법의 열쇠...,딩동 마법 세계 가야 택배 상자 지니 강 집 배달 신비롭다 마법 열쇠 지니 강 비밀...,"['딩동', '마법', '세계', '가야', '택배', '상자', '지니', '강'...","b""\xe1uF=`\x1c\xb3<\x01\x92\x93>\xfc&~\xbe|\xa...",[ 0.04845226 0.02186412 0.2882233 -0.248195...,"[-0.4505958557128906, 0.14335589110851288, 1.4...",오페라,마법,NaN
11984,150349,난타 대구,http://ticketimage.interpark.com/PlayDictionar...,퍼포먼스,2019/12/20 ~ 201

In [23]:
# 'musical_id' 컬럼을 기준으로 중복된 값 확인
duplicate_ids = past_data[past_data.duplicated(subset=['musical_id'], keep=False)]
print(duplicate_ids)

Empty DataFrame
Columns: [musical_id, title, poster_url, genre, date, start_date, end_date, location, actors, age_rating, running_time, describe, synopsis, synopsis_clear, tokenized_data, synopsis_vector, synopsis_numpy, synopsis_numpy_scale, tag1, tag2, tag3]
Index: []

[0 rows x 21 columns]


In [31]:
# 인덱스 재설정
past_data.reset_index(drop=True, inplace=True)
past_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,...,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector,synopsis_numpy,synopsis_numpy_scale,tag1,tag2,tag3
0,3885,실연남녀,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/10/13 ~ 2007/12/09,2007-10-13,2007-12-09,대학로 티오엠 1관,"엄기준,신성록,강지우,양소민,한애리,변우민,성민...",7세이상,...,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"['조폭', '형제', '경영', '아트', '퓨쳐', '산장', '이', '개월'...",b'\x0e\x11u<\xdeM1=\xa1kX=\xd11\x06\xbe?\x03\x...,[ 0.01495768 0.04328715 0.05283702 -0.131049...,"[-0.6805413961410522, 0.3137694299221039, -0.2...",사람,사랑,슬픔
1,3635,김종욱 찾기,http://ticketimage.interpark.com/PlayDictionar...,창작,2006/12/12 ~ 2007/04/08,2006-12-12,2007-04-08,JTN 아트홀 1관,"신성록,전병욱,오나라,안유진,원종환,김재범,김지현...",만 13세이상,...,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"['첫사랑', '나선', '여자', '첫사랑', '남자', '티격태격', '로맨틱'...",b'\xddC\xf5=I@Z=%\xfd\x8b\xbc\xd8\x03==\xd6q\x...,[ 0.11975835 0.05328396 -0.01708848 0.046146...,"[0.03893159702420235, 0.3932909667491913, -0.7...",사랑,오페라,NaN
2,4829,첫사랑,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/03/27 ~ 2007/06/17,2007-03-27,2007-06-17,대학로 티오엠 1관,"조정석,홍광호,해이,전경수,이정섭,김성기,김인수...",만 6세이상,...,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"['시간', '조금', '수', '있다', '적', '동네', '해수', '오누이'...","b""S\xa2\xa0<\xeb\xad\xd5<\xd6\x93\xbe\xbd\x01\...",[ 0.01960865 0.02608391 -0.09305541 -0.089630...,"[-0.6486117243766785, 0.176923006772995, -1.25...",오페라,NaN,NaN
3,3322,미라클,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/06/01 ~ 2013/02/17,2007-06-01,2013-02-17,미라클 씨어터 1관,"신현묵,조호균,김고운,원혜지,남궁인,윤도훈,김학규...",만 12세이상,...,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"['인기', '그룹', '핫바', '멤버', '가수', '희다', '은', '날',...",b'^\xb5D>\xb63\xad=P\x8e >.\x97\x02\xbe\xee+\x...,[ 1.92098111e-01 8.45712870e-02 1.56792879e-...,"[0.5355554223060608, 0.6421719193458557, 0.540...",오페라,NaN,NaN
4,4424,대장금,http://ticketimage.interpark.com/PlayDictionar...,창작,2007/05/26 ~ 2007/06/17,2007-05-26,2007-06-17,예술의전당 오페라극장,"최보영,안유진,김우형,김소현,이태원,류수화,손광업,한애리,한성식,김나윤,김일우,최윤...",만 7세이상,...,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"['어머니', '유언', '위해', '궁', '생각', '시가', '장금', '것'...",b'@\x8a\xa8=\x8e\xd7\xe0=\x1e\x10\x9b=N%\xf6\x...,[ 0.08229494 0.10978614 0.07571433 -0.120188...,"[-0.21826054155826569, 0.84274822473526, -0.04...",사랑,오페라,아이
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8878,150211,브레멘 음악대 부산,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2019/11/16 ~ 2019/11/17,2019-11-16,2019-11-17,신세계센텀시티 문화홀9층,NaN,24개월이상,...,NaN,집에서 키워지던 동물들이 주인의 천대를 받고쫓겨나거나 스스로의 자유를 위해 집을 나...,집 동물 주인 천대 스스로 자유 위해 집 친구 멋지다 악기 음악회 만들기 즐겁다 시...,"['집', '동물', '주인', '천대', '스스로', '자유', '위해', '집'...","b'e#^\xbc\xc5o\x10\xbd\x07,\x02>\xa4\xd4\x1c>\...",[-1.35582434e-02 -3.52628417e-02 1.27121076e-...,"[-0.8763076663017273, -0.3110712766647339, 0.3...",친구,예술,NaN
8879,150242,헤이지니 럭키강이 2 청주,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2019/12/28 ~ 2019/12/29,2019-12-28,2019-12-29,청주예술의전당 대공연장,NaN,24개월이상,...,NaN,딩동 마법의 세계로 가야할 택배상자가 지니강이 집으로 배달되었다신비로운 마법의 열쇠...,딩동 마법 세계 가야 택배 상자 지니 강 집 배달 신비롭다 마법 열쇠 지니 강 비밀...,"['딩동', '마법', '세계', '가야', '택배', '상자', '지니', '강'...","b""\xe1uF=`\x1c\xb3<\x01\x92\x93>\xfc&~\xbe|\xa...",[ 0.04845226 0.02186412 0.2882233 -0.248195...,"[-0.4505958557128906, 0.14335589110851288, 1.4...",오페라,마법,NaN
8880,150349,난타 대구,http://ticketimage.interpark.com/PlayDictionar...,퍼포먼스,2019/12/20 ~ 2019/1

In [33]:
# 현재 상영중 데이터를 데이터프레임으로 로드
present_data = pd.read_csv('../present_data/Data5_present_final.csv', encoding='utf-8')# 데이터 파일 경로
present_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,...,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector,synopsis_numpy,synopsis_numpy_scale,tag1,tag2,tag3
0,190722,레미제라블,http://ticketimage.interpark.com/PlayDictionar...,라이선스,2023/11/30 ~ 2024/03/10,2023-11-30,2024-03-10,블루스퀘어 신한카드홀,"민우혁,최재림,김우형,카이,조정은,린아,임기홍,육현욱,박준면,김영주,김성식,김진욱,...",NaN,...,Les Misérables\n뮤지컬 레미제라블\n시대를 뛰어넘은\n위대한 명작\n월...,사랑 용기 희망에 대한 대서사시\r\n기구한 운명의 청년 장발장이 빵을 훔친 죄로 ...,사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...,"['사랑', '용기', '희망', '대한', '서사시', '기구', '운명', '청...",b'\r\xc1\x90\xbb\x04\x8b\xd4;\x13\xfd :\xf2u\x...,[-0.00441754 0.0064863 0.00061412 0.002005...,"[1.4156763553619385, -0.8849654197692871, -0.1...",사랑,친구,NaN
1,190036,레베카,http://ticketimage.interpark.com/PlayDictionar...,라이선스,2023/08/19 ~ 2023/11/19,2023-08-19,2023-11-19,블루스퀘어 신한카드홀,"류정한,민영기,에녹,테이,신영숙,옥주현,리사,장은아,김보경,이지혜,이지수,웬디,윤석...",8세이상,...,더 이상 수식어가 필요 없는 가장 완벽한 뮤지컬역사적 10주년 기념공연으로 돌아온 ...,불의의 사고로 아내 레베카를 잃고 힘든 나날을 보내고 있는 막심 드 윈터그는 몬테카...,불의 사고 아내 레베카 힘드다 나날 있다 막심 드 윈터 그 몬테카를로 여행 중 사랑...,"['불의', '사고', '아내', '레베카', '힘드다', '나날', '있다', '...",b'\xd8M\xd1\xbb[\x8d\x14<\x18\x1f\xf09v\x01\x0...,[-6.38745353e-03 9.06690490e-03 4.57995338e-...,"[0.27278852462768555, 0.5870901346206665, -0.2...",사람,친구,사랑
2,183052,오페라의 유령,http://ticketimage.interpark.com/PlayDictionar...,라이선스,2023/07/21 ~ 2023/11/19,2023-07-21,2023-11-19,샤롯데씨어터,"조승우,최재림,김주택,전동석,손지수,송은혜,송원근,황건하,윤영석,이상준,이지영,한보...",초등학생이상 관람가,...,13년 간의 긴 기다림마침내 한국의 유령이 온다환영과도 같은 무대사라지지 않을 영원...,거부할수없는 강렬한 이끌림 가면속에 감춰진 아름다운 러브스토리 파리 오페라 하우...,거부 없다 강렬하다 림 가면 속 아름답다 러브스토리 파리 오페라 하우스 무명 무용수...,"['거부', '없다', '강렬하다', '림', '가면', '속', '아름답다', '...","b""\x9f\xf8\x93\xbb\x0b\x98\x00<6\xe5\x8a:\xb3t...",[-4.51572193e-03 7.84874987e-03 1.05968746e-...,"[1.358716607093811, -0.10778218507766724, 0.19...",공포,사랑,오페라
3,190718,벤허,http://ticketimage.interpark.com/PlayDictionar...,창작,2023/09/02 ~ 2023/11/19,2023-09-02,2023-11-19,LG아트센터 서울 LG SIGNATURE 홀,"박은태,신성록,규현,이지훈,박민성,서경수,윤공주,이정화,최지혜,이정열,홍경수,한지연...",8세이상,...,대한민국 창작 뮤지컬의 압도적 수작의 귀환제2회 한국뮤지컬어워즈 대상 무대예술상 앙...,서기 26년 제정 로마의 박해에 신음하는 예루살렘 명망 높은 유대의 귀족 벤허는 로...,서기 제정 로마 박해 신음 예루살렘 명망 높다 유대 귀족 벤허 로마 장교 친구 메 ...,"['서기', '제정', '로마', '박해', '신음', '예루살렘', '명망', '...",b'(\xca\xab\xbb\xf5\xc3\xc3;*H\xa68\x1c\xd5\x8...,[-5.24260476e-03 5.97428763e-03 7.92893843e-...,"[0.9369987845420837, -1.1770318746566772, -0.5...",사랑,친구,NaN
4,190723,레미제라블 부산,http://ticketimage.interpark.com/PlayDictionar...,라이선스,2023/10/11 ~ 2023/11/19,2023-10-11,2023-11-19,부산 드림씨어터,"민우혁,최재림,김우형,카이,조정은,린아,임기홍,육현욱,박준면,김영주,김성식,김진욱,...",만 7세이상,...,시대를 뛰어넘는 마스터피스10년 만의 위대한 귀환한국 라이선스 공연 10주년 기념역...,사랑 용기 희망에 대한 대서사시\r\n기구한 운명의 청년 장발장이 빵을 훔친 죄로 ...,사랑 용기 희망 대한 서사시 기구 운명 청년 장발장 빵 죄 감옥 후 가석방 전과자 ...,"['사랑', '용기', '희망', '대한', '서사시', '기구', '운명', '청...",b'\r\xc1\x90\xbb\x04\x8b\xd4;\x13\xfd :\xf2u\x...,[-0.00441754 0.0064863 0.00061412 0.002005...,"[1.4156763553619385, -0.8849654197692871, -0.1...",사랑,친구,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,195267,페인터즈,http://ticketimage.interpark.com/PlayDictionar...,퍼포먼스,2023/11/01 ~ 2023/12/31,2023-11-01,2023-12-31,명보아트홀,NaN,전체관람가,...,보고나면 사랑할 수 밖에 없는 4명의 페인터즈\n그들이 펼치는 환상의 미술 공연\n...,보고나면 사랑할 수 밖에 없는 4명의 페인터즈\n그들이 펼치는 환상의 미술 공연\n...,사랑 수 없다 명의 페인터 그 환상 미술 공연 시즌 놀랍다 라이브 드로잉 감각 어 ...,"['사랑', '수', '없다', '명의', '페인터', '그', '환상', '미술'...",b'b1\xda\xbb\xaf\xcb\x03<\xfa\xdd\x82\xbaWw9;\...,[-0.00665872 0.00804417 -0.00099844 0.002829...,"[0.11540764570236206, 0.0036898653488606215, -...",사랑,친구,NaN
81,167486,장수탕 선녀님,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2021/09/16 ~ 2024/03/03,2021-09-16,2024-03-03,서울숲 씨어터 2관,"한송이,박시연,한리예,최승환,최혜인,박시온,양다은,이서은,최성애,김인섭,이용호,김명...",12개월이상,...,가족뮤지컬 연간 판매랭킹 1위 동네 낡은 목욕탕에서 만나는 신비로운 판타지의 세계...,겁먹지 마라 얘야 나는 저기 산속에 사는 선녀란다 우리 동네에는 오래된 목욕탕이...,겁 마르다 얘 나 저기 산속 선녀 란 우리 동네 오래되다 목욕탕 있다 큰길 새롭다 ...,"['겁', '마르다', '얘', '나', '저기', '산속', '선녀', '란', ...",b'\xc3}\xd0\xbb\x99\xe4\xc8;\xfe\xe3\x95\xb9\x...,[-6.

#### past_data

In [34]:
# NaN 값을 빈 리스트로 대체
past_data['synopsis_numpy_scale'].fillna('[]', inplace=True)

/tmp/ipykernel_21150/1729146996.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  past_data['synopsis_numpy_scale'].fillna('[]', inplace=True)


In [35]:
import ast
# 데이터프레임에서 synopsis_numpy_scale 열의 값을 파싱하여 리스트로 변환
past_data['synopsis_numpy_scale'] = past_data['synopsis_numpy_scale'].apply(ast.literal_eval)

/tmp/ipykernel_21150/3944376945.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  past_data['synopsis_numpy_scale'] = past_data['synopsis_numpy_scale'].apply(ast.literal_eval)


In [38]:
# 'synopsis_numpy_scale' 열에서 누락된 값(null 또는 NaN) 확인
missing_values = past_data['synopsis_numpy_scale'].isnull().sum()
print("누락된 값의 개수:", missing_values)

# 'synopsis_numpy_scale' 열의 길이가 0인 배열 확인
zero_length_arrays = past_data[past_data['synopsis_numpy_scale'].apply(len) == 0]
print("길이가 0인 배열의 개수:", len(zero_length_arrays))
zero_length_arrays

누락된 값의 개수: 0
길이가 0인 배열의 개수: 5


,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,...,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector,synopsis_numpy,synopsis_numpy_scale,tag1,tag2,tag3
867,7962,난타 강남공연,http://ticketimage.interpark.com/PlayDictionar...,창작,2009/01/01 ~ 2010/06/30,2009-01-01,2010-06-30,우림 청담 씨어터,"남동훈,이와모토 유카,나준석,이대진,권혁진,황요한,이한범...",36개월이상,...,난타는 한국 전통 가락인 사물놀이 리듬을 소재로 주방에서 일어나는 일들을 코믹하게 ...,주방장을 비롯한 세 명의 요리사가 하루일과를 시작한다 세명의 요리사가 주방에서 야채...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN
2670,30940,부석사,http://ticketimage.interpark.com/PlayDictionar...,창작,2011/12/17 ~ 2011/12/17,2011-12-17,2011-12-17,영주 국민체육센터,NaN,NaN,...,의상과 선묘수행을 통해 사랑을 깨닫다,전쟁 중에 부모를 잃은 선묘는 당나라에 공출이 되어 끌려가고 월산은 신라의 사노비가...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN
3331,28142,오즈의 마법사 광주,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2011/10/22 ~ 2011/10/22,2011-10-22,2011-10-22,광산구 청소년수련관,NaN,초등학생이상 관람가,...,NaN,캔자스주에서 아저씨 아주머니와 함께 살고 있던 소녀 도로시,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN
3602,31828,톰과 제리 김해 진영,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2012/01/29 ~ 2012/01/29,2012-01-29,2012-01-29,진영한빛도서관 공연장,NaN,전체관람가,...,시대를 초월한 워너브라더스사의 애니메이션 톰과 제리가 캐릭터 뮤지컬로 어린이를 찾아...,말썽꾸러기인 톰에게 제리가 더 말썽꾸러기 인가 보네요,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN
4196,31768,톰과 제리 김해 장유,http://ticketimage.interpark.com/PlayDictionar...,어린이가족,2012/01/28 ~ 2012/01/28,2012-01-28,2012-01-28,김해장유문화센터 공연장,NaN,전체관람가,...,시대를 초월한 워너브라더스사의 애니메이션 톰과 제리가 캐릭터 뮤지컬로 어린이를 찾아...,말썽꾸러기인 톰에게 제리가 더 말썽꾸러기 인가 보네요,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN


In [39]:
# 길이가 0인 배열을 포함한 행 삭제
past_data = past_data.drop(zero_length_arrays.index)

# 변경된 데이터 확인
print(past_data.shape)  # 변경된 데이터프레임의 모양 출력

(8878, 21)


In [40]:
# StandardScaler를 사용하여 특성들을 표준 스케일링
scaler_past = StandardScaler()
past_data_scaled = scaler_past.fit_transform(np.vstack(past_data['synopsis_numpy_scale']))
# 음수 값을 양수로 변환
past_data_scaled = past_data_scaled - np.min(past_data_scaled) + 1e-10  # 최소값에 아주 작은 값을 더하여 0이 되지 않도록 함

#### present_data

In [41]:
# 데이터프레임에서 synopsis_numpy_scale 열의 값을 파싱하여 리스트로 변환
present_data['synopsis_numpy_scale'] = present_data['synopsis_numpy_scale'].apply(ast.literal_eval)

In [42]:
# StandardScaler를 사용하여 특성들을 표준 스케일링
scaler_present = StandardScaler()
present_data_scaled = scaler_present.fit_transform(np.vstack(present_data['synopsis_numpy_scale']))
# 음수 값을 양수로 변환
present_data_scaled = present_data_scaled - np.min(present_data_scaled) + 1e-10  # 최소값에 아주 작은 값을 더하여 0이 되지 않도록 함

### 유사한 작품 추천

In [55]:
# 과거 데이터에서 작품을 선택 (예: musical_id가 3885인 작품)
selected_work_index_past = past_data[past_data['musical_id'] == 4424].index[0]

In [56]:
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

# NMF 모델 초기화
nmf = NMF(n_components=10, init='random', random_state=42)

# 특성 행렬 생성
V = np.vstack(past_data_scaled)

# NMF 모델 훈련
W = nmf.fit_transform(V)
H = nmf.components_

# 현재 상영중인 데이터에 대한 특성 행렬 생성
V_present = np.vstack(present_data_scaled)

# NMF 모델을 사용하여 현재 상영중인 데이터의 특성 행렬 분해
W_present = nmf.transform(V_present)

# 선택한 작품과 다른 작품 간의 코사인 유사도 계산
selected_work = W[selected_work_index_past].reshape(1, -1)
similarities = cosine_similarity(W_present, selected_work)

# 유사도가 높은 순서대로 정렬하여 유사한 작품 인덱스를 찾습니다.
similar_work_indices = similarities.argsort(axis=0)[::-1].flatten()
top_n = 5

print("선택한 작품:")
print(past_data.loc[selected_work_index_past, 'title'])
print(f"Musical ID: {past_data.loc[selected_work_index_past, 'musical_id']}")
print("")

for i in range(1, top_n + 1):
    index = similar_work_indices[i]
    similarity = similarities[index]
    title = present_data.loc[index, 'title']
    musical_id = present_data.loc[index, 'musical_id']
    print(f"유사한 작품: {title}, Musical ID: {musical_id}, 정규화된 유사도: {similarity}")

선택한 작품:
대장금
Musical ID: 4424

유사한 작품: 셰프, Musical ID: 191356, 정규화된 유사도: [0.97881727]
유사한 작품: 시스터즈, Musical ID: 191013, 정규화된 유사도: [0.97850121]
유사한 작품: 당신만이, Musical ID: 174445, 정규화된 유사도: [0.97650897]
유사한 작품: 인사이드미, Musical ID: 194128, 정규화된 유사도: [0.97645473]
유사한 작품: 오즈의 마법사, Musical ID: 187126, 정규화된 유사도: [0.97560916]


/home/user1/.local/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1710: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
